### Troubleshooting: Set working directory

In [6]:
# import os 
# print("starting directory: ", os.getcwd())
# os.chdir("/w/halld-scshelf2101/home/dbarton/gluex/analysis_KpiLambda")
# print("changed directory to: ", os.getcwd())
# !source /u/home/dbarton/.cshrc
# !source /group/halld/Software/build_scripts/gluex_env_jlab.csh

# import ROOT

### Startup FSRoot environment

In [ ]:
# To use FSRoot in Jupyter Notebooks you need to be in a special container per  https://lan13005.github.io/PyAmpTools/intro.html#apptainer-usage

# STEP 1: Open a terminal on ifarm

# STEP 2: Initialize the container using this alias (defined here below)
# $ fshell ######## This outputs a "localhost" URL.

# STEP 3: Select your kernel in VS Code
    # Open new Jupyter kernel --> copy & paste URL from STEP 2 --> ENTER

# Note: you can run the alias directly in a Jupyter Notebook cell with '!fshell'.  But it often crashes.  Just do it in a terminal.

# Alias definition in STEP 2:
# Add these lines to your shell configuration file (~/.cshrc for tcsh):
# alias fshell 'apptainer exec --contain --writable-tmpfs -B /w/halld-scshelf2101/home/dbarton/gluex,/scratch --env BASH_ENV=/dev/null /w/halld-scshelf2101/lng/WORK/PyAmpTools9/pyamptools.sif bash -c "source /etc/bash.bashrc; jupyter-lab --no-browser --port=8888; exec bash"'

In [1]:
import ROOT
from pyamptools import atiSetup
import os

os.chdir("/w/halld-scshelf2101/home/dbarton/gluex/analysis_KpiLambda")
os.listdir()

atiSetup.setup(globals(), use_fsroot=True)

# fileName = '/w/halld-scshelf2101/home/dbarton/gluex/dselectors/pi0kplamb_flat_2018_08.root'
fileName = '/w/halld-scshelf2101/home/dbarton/gluex/analysis_KpiLambda/pi0kplamb_FSflat_2018_08.root'
treeName = 'pi0kplamb_flat'

Welcome to JupyROOT 6.28/06
atiSetup| jupyter-lab called python3.9

------------------------------------------------
atiSetup| MPI is disabled
atiSetup| GPU is disabled
------------------------------------------------


atiSetup| Loading library libIUAmpTools.so ............  ON
atiSetup| Loading library libAmpTools.so ..............  ON
atiSetup| Loading library libAmpPlotter.so ............  ON
atiSetup| Loading library libAmpsDataIO.so ............  ON
atiSetup| Loading library libFSRoot.so ................  ON
atiSetup| Loading library libAmpsGen.so ...............  OFF


------------------------------------------------
------------------------------------------------



### Define cuts

In [8]:
# coherent peak cut 8.2 - 8.8

<p style="text-align: center;">
<u style="font-size:12px;">_____________</u><br>
<u style="font-size:18px;">_____________________</u><br>
<u style="font-size:24px;">___________________________</u>
</p>

<h2 style="text-align: center;">PLOTS</h2>

<p style="text-align: center;">
<u style="font-size:24px;">___________________________</u><br>
<u style="font-size:18px;">_____________________</u><br>
<u style="font-size:12px;">_____________</u>
</p>

### List contents of ROOT file

In [2]:
def list_trees_and_branches(fileName):
    file = ROOT.TFile.Open(fileName)
    # Loop over all keys in the file
    for key in file.GetListOfKeys():
        obj = key.ReadObj()
        # Check if the object is a tree
        if isinstance(obj, ROOT.TTree):
            tree = obj
            print(f"Tree Name: {tree.GetName()}")
            print("Branches:")
            # List all branches in the tree
            branches = tree.GetListOfBranches()
            for branch in branches:
                print(f" - {branch.GetName()}")

    # Close the file
    file.Close()

list_trees_and_branches(fileName)

Tree Name: ntFSGlueX_100000000_100001
Branches:
 - Run
 - Event
 - Chi2
 - Chi2DOF
 - RFTime
 - RFDeltaT
 - EnUnusedSh
 - NumUnusedTracks
 - NumNeutralHypos
 - NumBeam
 - NumCombos
 - ProdVx
 - ProdVy
 - ProdVz
 - ProdVt
 - PxPB
 - PyPB
 - PzPB
 - EnPB
 - VxPB
 - VyPB
 - VzPB
 - RPxPB
 - RPyPB
 - RPzPB
 - REnPB
 - RVxPB
 - RVyPB
 - RVzPB
 - PxP1
 - PyP1
 - PzP1
 - EnP1
 - RPxP1
 - RPyP1
 - RPzP1
 - REnP1
 - VeeLP1
 - VeeLSigmaP1
 - PxP1a
 - PyP1a
 - PzP1a
 - EnP1a
 - VxP1a
 - VyP1a
 - VzP1a
 - RPxP1a
 - RPyP1a
 - RPzP1a
 - REnP1a
 - RVxP1a
 - RVyP1a
 - RVzP1a
 - TkNDFP1a
 - TkChi2P1a
 - TkTOFBetaP1a
 - TkTOFChi2P1a
 - TkDEDXChi2P1a
 - TkDEDXNDFP1a
 - TkDEDXCDCP1a
 - TkDEDXFDCP1a
 - PxP1b
 - PyP1b
 - PzP1b
 - EnP1b
 - VxP1b
 - VyP1b
 - VzP1b
 - RPxP1b
 - RPyP1b
 - RPzP1b
 - REnP1b
 - RVxP1b
 - RVyP1b
 - RVzP1b
 - TkNDFP1b
 - TkChi2P1b
 - TkTOFBetaP1b
 - TkTOFChi2P1b
 - TkDEDXChi2P1b
 - TkDEDXNDFP1b
 - TkDEDXCDCP1b
 - TkDEDXFDCP1b
 - PxP2
 - PyP2
 - PzP2
 - EnP2
 - VxP2
 - VyP2
 - VzP2
 

### Beam Energy

In [ ]:
from IPython.display import Image, display

plotTitle = 'Beam energy'
plot2Title = 'Beam energy measured'
axisLabel = 'Beam energy [GeV]'

canvas = ROOT.TCanvas('c', 'c', 800, 600)
    
h1 = ROOT.FSHistogram.getTH1F(fileName, treeName, "beam_e", "(80, 0.0, 14.0)", "")
h1.SetStats(False)
h1.SetLineColor(ROOT.kBlue)
h1.GetXaxis().SetTitle(axisLabel)
h1.GetYaxis().SetTitle('Events')
h1.Draw()

h2 = ROOT.FSHistogram.getTH1F(fileName, treeName, "beam_e_measured", "(80, 0.0, 14.0)", "")
h2.SetStats(False)
h2.SetMarkerStyle(24)
h2.SetMarkerColor(ROOT.kRed)
h2.GetXaxis().SetTitle(axisLabel)
h2.GetYaxis().SetTitle('Events')
h2.Draw('p same')
    
legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.75)
# legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
# legend1.AddEntry(h2.GetValue(), plot2Title, 'p')
legend1.Draw('same')

img_path = 'plots/pi0kplamb_beam_e.png'
canvas.SaveAs(img_path)
canvas.Close()
display(Image(filename=img_path))

### Target proton

In [10]:
# from IPython.display import Image, display

# plotTitle = 'Target proton mass'
# axisLabel = 'Mass (target proton) [GeV/c^{2}]'

# def plot_hist(df):

#     dfFiltered = df

#     canvas = ROOT.TCanvas("c", "c", 800, 600)
#     h1 = dfFiltered.Histo1D(("h1", plotTitle, 60, 0, 3), '??????')
#     h1.SetStats(False)
#     h1.SetLineColor(ROOT.kBlue)
#     h1.GetXaxis().SetTitle(axisLabel)
#     h1.GetYaxis().SetTitle('Events')
#     h1.Draw()
#     legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
#     legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
#     legend1.Draw('same')

#     img_path = '../_plots/pi0kplamb_target_m.png'
#     canvas.SaveAs(img_path)
#     canvas.Close()
#     display(Image(filename=img_path))

# plot_hist(dfpi0kplamb)

### $K^{+}$ $\pi^{0}$ $\pi^{-}$

In [ ]:
from IPython.display import Image, display

plotTitle = '$\gamma p -> (K^{+} \pi^{0} \pi^{-})$ [GeV/c^{2}]'
axisLabel = '$Mass (K^{+} \pi^{0} \pi^{-})$ [GeV/c^{2}]'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    h1 = dfFiltered.Histo1D(('h1', plotTitle, 60, 0.05, 2.0), 'kpPi0Pim_m')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_kpPi0Pim_m.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### $K^{+} \pi^{0}$ (can be $K^{*}$)

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt

plotTitle = '$Mass (K^{*}? -> K^{+} \pi^{0})$ [GeV/c^{2}]'
axisLabel = '$Mass (K^{+} \pi^{0})$ [GeV/c^{2}]'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    h1 = dfFiltered.Histo1D(('h1', plotTitle, 60, 0.05, 2.0), 'kpPi0_m')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_kpPi0_m.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### $K^{+}$ mass

In [ ]:
from IPython.display import Image, display

plotTitle = 'K^{+} Mass'
plot2Title = 'K^{+} Mass Measured'
axisLabel = 'Mass (K^{+}) [GeV/c^{2}]'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    h1 = dfFiltered.Histo1D(('h1', plotTitle, 60, 0.45, 0.55), 'kp_m')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()

    h2 = dfFiltered.Histo1D(('h2', plot2Title, 60, 0.45, 0.55), 'kp_m_measured')
    h2.SetStats(False)
    h2.SetMarkerStyle(24)
    h2.SetMarkerColor(ROOT.kRed)
    # h2.GetXaxis().SetTitle(axisLabel)
    # h2.GetYaxis().SetTitle('Events')
    h2.Draw('p same')

    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.AddEntry(h2.GetValue(), plot2Title, 'p')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_kp_m.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### $\pi_{0}$ mass

In [ ]:
from IPython.display import Image, display

plotTitle = '$\pi^{0}$ Mass'
axisLabel = '$Mass (\pi^{0})$ [GeV/c^{2}]'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    h1 = dfFiltered.Histo1D(('h1', plotTitle, 60, 0.05, 0.2), 'decayingPi0_m')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_pi0_m.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### $\Lambda$ mass

In [ ]:
from IPython.display import Image, display

plotTitle = '$\Lambda$ Mass'
axisLabel = '$Mass (p \pi^{-}) [GeV/c^{2}]$'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    h1 = dfFiltered.Histo1D(('h1', plotTitle, 120, 1.06, 1.3), 'decayingLambda_m')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_lambda_m.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### $\gamma_{1}$ energy

In [ ]:
from IPython.display import Image, display

plotTitle = '$\gamma_{1}$ Energy'
plot2Title = '$\gamma_{1}$ Momentum'
axisLabel = '$Energy (\gamma_{1}) \ [GeV]$'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    
    h1 = dfFiltered.Histo1D(('photon1_E', plotTitle, 120, -1.0, 6.0), 'photon1_E')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()

    h2 = dfFiltered.Histo1D(('photon1_p', plot2Title, 120, -1.0, 6.0), 'photon1_p')
    h2.SetStats(False)
    h2.SetMarkerStyle(24)
    h2.SetMarkerColor(ROOT.kRed)
    h2.GetXaxis().SetTitle(axisLabel)
    h2.GetYaxis().SetTitle('Events')
    h2.Draw('p same')
    
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.75)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.AddEntry(h2.GetValue(), plot2Title, 'p')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_photon1_E.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### $\gamma_{2}$ energy

In [ ]:
from IPython.display import Image, display

plotTitle = '$\gamma_{2}$ Energy'
plot2Title = '$\gamma_{2}$ Momentum'
axisLabel = '$Energy (\gamma_{2}) \ [GeV]$'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    
    h1 = dfFiltered.Histo1D(('photon1_E', plotTitle, 120, -1.0, 6.0), 'photon2_E')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()

    h2 = dfFiltered.Histo1D(('photon1_p', plot2Title, 120, -1.0, 6.0), 'photon2_p')
    h2.SetStats(False)
    h2.SetMarkerStyle(24)
    h2.SetMarkerColor(ROOT.kRed)
    h2.GetXaxis().SetTitle(axisLabel)
    h2.GetYaxis().SetTitle('Events')
    h2.Draw('p same')
    
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.75)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.AddEntry(h2.GetValue(), plot2Title, 'p')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_photon1_E.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### Recoiling proton Mass

In [ ]:
from IPython.display import Image, display

plotTitle = 'Proton Mass'
axisLabel = 'Mass (proton reoil) [GeV/c^{2}]'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    h1 = dfFiltered.Histo1D(('h1', plotTitle, 120, 0.9, 1.0), 'protonRecoil_m')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_protonRecoil_m.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### $\pi_{-}$ mass

In [ ]:
from IPython.display import Image, display

plotTitle = '$\pi^{-}$ Mass'
axisLabel = '$Mass (\pi^{-}) [GeV/c^{2}]$'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)
    h1 = dfFiltered.Histo1D(('h1', plotTitle, 120, 0.1, 0.18), 'pim_m')
    h1.SetStats(False)
    h1.SetLineColor(ROOT.kBlue)
    h1.GetXaxis().SetTitle(axisLabel)
    h1.GetYaxis().SetTitle('Events')
    h1.Draw()
    legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    legend1.Draw('same')

    img_path = 'plots/pi0kplamb_pim_m.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

<p style="text-align: center;">
<u style="font-size:12px;">_____________</u><br>
<u style="font-size:18px;">_____________________</u><br>
<u style="font-size:24px;">___________________________</u>
</p>

<h2 style="text-align: center;">VARIOUS COMBINATIONS</h2>

<p style="text-align: center;">
<u style="font-size:24px;">___________________________</u><br>
<u style="font-size:18px;">_____________________</u><br>
<u style="font-size:12px;">_____________</u>
</p>

### Dalitz plot: ($p_{K^{+}} + p_{\pi^{0}})^{2}$ vs. ($p_{\Lambda} + p_{\pi^{0}})^{2}$

In [ ]:
from IPython.display import Image, display

plotTitle = 'Dalitz plot'
XaxisLabel = '$({K^{+}} + {\pi^{0}})^{2} \ [GeV^{2}/c^{4}]$'
YaxisLabel = '$({\Lambda} + {\pi^{0}})^{2} \ [GeV^{2}/c^{4}]$'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)

    h1 = dfFiltered.Histo2D(('DalitzPlot', 'DalitzPlot', 80, 0.5, 4.5, 80, 0.5, 4.5), 'kpANDPi0_m', 'pi0ANDlambda_m')

    h1.SetStats(False)
    h1.GetXaxis().SetTitle(XaxisLabel)
    h1.GetYaxis().SetTitle(YaxisLabel)
    h1.Draw('colz')


    # legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    # legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    # legend1.Draw('same')

    img_path = 'plots/pi0kplamb_DALITZ_kpPi0VSlambPi0.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)

### Dalitz plot: ($p_{K^{+}} + p_{\Lambda})$ vs. ($p_{\pi^{0}} + p_{\Lambda})^{2}$

In [ ]:
from IPython.display import Image, display

plotTitle = 'Dalitz plot'
XaxisLabel = '$({K^{+}} + {\Lambda})^{2} \ [GeV^{2}/c^{4}]$'
YaxisLabel = '$({\pi^{0}} + {\Lambda})^{2} \ [GeV^{2}/c^{4}]$'

def plot_hist(df):

    dfFiltered = df

    canvas = ROOT.TCanvas('c', 'c', 800, 600)

    h1 = dfFiltered.Histo2D(('DalitzPlot', 'DalitzPlot', 80, 1.0, 4.7, 80, 1.0, 4.7), 'kpANDlambda_m', 'pi0ANDlambda_m')

    h1.SetStats(False)
    h1.GetXaxis().SetTitle(XaxisLabel)
    h1.GetYaxis().SetTitle(YaxisLabel)
    h1.Draw('colz')


    # legend1 = ROOT.TLegend(0.7, 0.9, 0.9, 0.85)
    # legend1.AddEntry(h1.GetValue(), plotTitle, 'l')
    # legend1.Draw('same')

    img_path = 'plots/pi0kplamb_DALITZ_kpLambVSPi0Lamb.png'
    canvas.SaveAs(img_path)
    canvas.Close()
    display(Image(filename=img_path))

plot_hist(dfpi0kplamb)